# Robots

### Overview
We provide a wide variety of **Robots** that can be imported into the **Simulator**.

| Agent Name     | DOF | Information      | Controller |
|:-------------: | :-------------: |:-------------: |:-------------|
| Mujoco Ant      | 8     | [OpenAI Link](https://blog.openai.com/roboschool/) | Torque |
| Mujoco Humanoid | 17    | [OpenAI Link](https://blog.openai.com/roboschool/) | Torque |
| Husky Robot     | 4     | [ROS](http://wiki.ros.org/Robots/Husky), [Manufacturer](https://www.clearpathrobotics.com/) | Torque, Velocity, Position |
| Minitaur Robot  | 8     | [Robot Page](https://www.ghostrobotics.io/copy-of-robots), [Manufacturer](https://www.ghostrobotics.io/) | Sine Controller |
| Quadrotor       | 6     | [Paper](https://repository.upenn.edu/cgi/viewcontent.cgi?referer=https://www.google.com/&httpsredir=1&article=1705&context=edissertations) | Torque |
| TurtleBot       | 2     | [ROS](http://wiki.ros.org/Robots/TurtleBot), [Manufacturer](https://www.turtlebot.com/) | Torque, Velocity, Position, Differential Drive |
| Freight         | 2     | [Fetch Robotics Link](https://fetchrobotics.com/robotics-platforms/freight-base/) | Torque, Velocity, Position, Differential Drive|
| Fetch           | 10    | [Fetch Robotics Link](https://fetchrobotics.com/robotics-platforms/freight-base/) | Torque, Velocity, Position, Differential Drive |
| JackRabbot      | 2 & 7 | [Stanford Project Link](http://cvgl.stanford.edu/projects/jackrabbot/) | Torque, Velocity, Position, Differential Drive |
| LocoBot         | 2     | [ROS](http://wiki.ros.org/locobot), [Manufacturer](https://www.trossenrobotics.com/locobot-pyrobot-ros-rover.aspx) | Torque, Velocity, Position, Differential Drive |

Typically, these robot classes take in the URDF file or MuJoCo XML file of an robot (in `gibson2.assets_path`) and provide a `load` function that be invoked externally (usually by `import_robot` of `Simulator`). The `load` function imports the robot into PyBullet.

All robot clases inherit `LocomotorRobot`. Some useful functions are worth pointing out:
- `{get/set}_{position/orientation/rpy/linear_velocity/angular_velocity}`: get and set the physical states of the robot base
- `apply_robot_action`: set motor control for each of the controllable joints. It currently supports four modes of control: joint torque, velocity, position, and differential drive for two-wheeled robots
- `calc_state`: compute robot states that might be useful for external applications
- `robot_specific_reset`: reset the robot joint states to their default value, particularly useful for mobile manipulators. For instance, `Fetch.robot_specific_reset()` will reset the robot to be something like this:

![fetch.png](https://robots.ieee.org/robots/fetch/Photos/SD/fetch-photo1-full.jpg)

Here are some details about how we perform motor control for robots:
- Say the robot uses joint velocity control `self.control == 'velocity'`
- We assume the external user (say an RL agent) will call `apply_action` with `policy_action` that is always between `-1` and `1`.
- `policy_action` will be scaled to `robot_action` by `policy_action_to_robot_action` based on the action space. The action space is set by `config['velocity']` in the YAML config file
- `robot_action` will be applied by `apply_robot_action`, which internally executes the following:
```python
def apply_robot_action(action):
    for n, j in enumerate(self.ordered_joints):
        j.set_motor_velocity(self.velocity_coef * j.max_velocity * float(np.clip(action[n], -1, +1)))
```
Note that `robot_action` is a normalized joint velocity, i.e. `robot_action[n] == 1.0` means executing the maximum joint velocity for the nth joint. The limits of joint position, velocity and torque are extracted from the URDF file of the robot.

Most of the code can be found here: [gibson2/robots](https://github.com/StanfordVL/iGibson/blob/master/gibson2/robots).


## Examples

## Turtlebot

Control the velocity of turtlebot

In [1]:
from gibson2.robots.turtlebot_robot import Turtlebot
from gibson2.simulator import Simulator
from gibson2.scenes.gibson_indoor_scene import StaticIndoorScene
from gibson2.objects.ycb_object import YCBObject
from gibson2.utils.utils import parse_config
from gibson2.render.mesh_renderer.mesh_renderer_settings import MeshRendererSettings
import numpy as np
from gibson2.render.profiler import Profiler
import gibson2
import os
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

INFO:root:Importing iGibson (gibson2 module)
INFO:root:Assets path: /opt/igibson/gibson2/data/assets
INFO:root:Gibson Dataset path: /opt/igibson/gibson2/data/g_dataset
INFO:root:iG Dataset path: /opt/igibson/gibson2/data/ig_dataset
INFO:root:3D-FRONT Dataset path: /opt/igibson/gibson2/data/threedfront_dataset
INFO:root:CubiCasa5K Dataset path: /opt/igibson/gibson2/data/cubicasa_dataset
INFO:root:Example path: /opt/igibson/gibson2/examples
INFO:root:Example config path: /opt/igibson/gibson2/examples/configs


torch is not available, falling back to rendering to memory(instead of tensor)


In [15]:
config = parse_config(os.path.join(gibson2.example_config_path, 'turtlebot_demo.yaml'))
settings = MeshRendererSettings(enable_shadow=False, msaa=False)
s = Simulator(mode='headless', image_width=256,
              image_height=256, rendering_settings=settings)

scene = StaticIndoorScene('Rs',
                          build_graph=True,
                          pybullet_load_texture=True)
s.import_scene(scene)
turtlebot = Turtlebot(config)
s.import_robot(turtlebot)

turtlebot.set_position([0,0,0.1])

for i in range(100):
    s.step()
print(s.renderer.instances)


INFO:root:Device 0 is available for rendering
INFO:root:Using device 0 for rendering
INFO:root:IndoorScene model: Rs
INFO:root:StaticIndoorScene scene: Rs


******************PyBullet Logging Information:
PyBullet Logging Information******************


INFO:root:Loading traversable graph
INFO:root:Loading /opt/igibson/gibson2/data/g_dataset/Rs/mesh_z_up.obj
INFO:root:Loading robot model file: turtlebot/turtlebot.urdf
INFO:root:Loading /opt/igibson/gibson2/data/assets/models/turtlebot/kobuki_description/meshes/main_body.obj
INFO:root:Loading /opt/igibson/gibson2/data/assets/models/mjcf_primitives/cube.obj
INFO:root:Loading /opt/igibson/gibson2/data/assets/models/turtlebot/kobuki_description/meshes/wheel.obj
INFO:root:Loading /opt/igibson/gibson2/data/assets/models/mjcf_primitives/cube.obj
INFO:root:Loading /opt/igibson/gibson2/data/assets/models/mjcf_primitives/cube.obj
INFO:root:Loading /opt/igibson/gibson2/data/assets/models/turtlebot/turtlebot_description/meshes/stacks/hexagons/pole_bottom.obj
INFO:root:Loading /opt/igibson/gibson2/data/assets/models/turtlebot/turtlebot_description/meshes/stacks/hexagons/plate_bottom.obj
INFO:root:Loading /opt/igibson/gibson2/data/assets/models/turtlebot/turtlebot_description/meshes/stacks/hexagons

[Instance(0) -> Object(0), Robot(1) -> Objects(1,2,3,3,4,5,6,6,6,6,6,6,7,8,8,8,8,9,10,10,10,10,11,11,12,13)]


In [19]:
def show_camera_image(v_left, v_right):
    turtlebot.apply_action([v_left, v_right])
    for i in range(5):
        s.step()
    camera_pose = np.array([0.5, 0.5, 0.5]) + turtlebot.get_position()
    view_direction = np.array([-1, -1, -1])
    s.renderer.set_camera(camera_pose, camera_pose + view_direction, [0, 0, 1])
    frames = s.renderer.render(modes=('rgb'))[0]
    plt.imshow((frames[:,:,:3] * 255).astype(np.uint8))
    
interactive_plot = interact_manual(show_camera_image, v_left = (-1.0, 1.0), v_right = (-1.0, 1.0))
interactive_plot    

interactive(children=(FloatSlider(value=0.0, description='v_left', max=1.0, min=-1.0), FloatSlider(value=0.0, …

<function __main__.show_camera_image(v_left, v_right)>

## Fetch 
Control the base and arm of Fetch

In [ ]:
from gibson2.robots.fetch_robot import Fetch

config = parse_config(os.path.join(gibson2.example_config_path, 'turtlebot_demo.yaml'))
settings = MeshRendererSettings(enable_shadow=False, msaa=False)
s = Simulator(mode='headless', image_width=256,
              image_height=256, rendering_settings=settings)

scene = StaticIndoorScene('Rs',
                          build_graph=True,
                          pybullet_load_texture=True)
s.import_scene(scene)
fetch = Fetch(config)
s.import_robot(fetch)

fetch.set_position([0,0,0.1])

for i in range(100):
    s.step()
print(s.renderer.instances)


INFO:root:Device 0 is available for rendering
INFO:root:Using device 0 for rendering
INFO:root:IndoorScene model: Rs
INFO:root:StaticIndoorScene scene: Rs


******************PyBullet Logging Information:
PyBullet Logging Information******************


INFO:root:Loading traversable graph
INFO:root:Loading /opt/igibson/gibson2/data/g_dataset/Rs/mesh_z_up.obj
INFO:root:Loading robot model file: fetch/fetch.urdf
INFO:root:Loading /opt/igibson/gibson2/data/assets/models/fetch/meshes/base_link.obj
INFO:root:Loading /opt/igibson/gibson2/data/assets/models/fetch/meshes/r_wheel_link.obj
INFO:root:Loading /opt/igibson/gibson2/data/assets/models/fetch/meshes/l_wheel_link.obj
INFO:root:Loading /opt/igibson/gibson2/data/assets/models/fetch/meshes/torso_lift_link.obj
INFO:root:Loading /opt/igibson/gibson2/data/assets/models/fetch/meshes/head_pan_link.obj
INFO:root:Loading /opt/igibson/gibson2/data/assets/models/fetch/meshes/head_tilt_link.obj
INFO:root:Loading /opt/igibson/gibson2/data/assets/models/fetch/meshes/shoulder_pan_link.obj
INFO:root:Loading /opt/igibson/gibson2/data/assets/models/fetch/meshes/shoulder_lift_link.obj
INFO:root:Loading /opt/igibson/gibson2/data/assets/models/fetch/meshes/upperarm_roll_link.obj
INFO:root:Loading /opt/igibs

In [23]:
def show_camera_image(v_left, v_right):
    fetch.apply_action([v_left, v_right] + [0] * 8)
    for i in range(5):
        s.step()
    camera_pose = np.array([0.5, 0.5, 0.5]) + fetch.get_position()
    view_direction = np.array([-1, -1, -1])
    s.renderer.set_camera(camera_pose, camera_pose + view_direction, [0, 0, 1])
    frames = s.renderer.render(modes=('rgb'))[0]
    plt.imshow((frames[:,:,:3] * 255).astype(np.uint8))
    
interactive_plot = interact_manual(show_camera_image, v_left = (-0.3, 0.3), v_right = (-0.3, 0.3))
interactive_plot    

interactive(children=(FloatSlider(value=0.0, description='v_left', max=0.3, min=-0.3), FloatSlider(value=0.0, …

<function __main__.show_camera_image(v_left, v_right)>